In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json

In [2]:
with open('sample-data.json', mode = 'r', encoding = 'utf-8') as file:
    data = json.load(file)

In [3]:
vstup = pd.DataFrame(data)

In [4]:
vystup = pd.DataFrame(columns = ['name', 'cpu_usage', 'memory_usage', 'created_at', 'status', 'IP'])

In [5]:
#name, cpu a memory usage, created_at, status a všechny přiřazené IP adresy. Datumová pole převeďte na UTC timestamp.

In [6]:
vystup.name = vstup.name

In [7]:
#cpu usage
for i,e in enumerate(vstup.state):
    try:
        vystup.cpu_usage[i] = vstup.state[i]['cpu']['usage']
    except:
        pass

In [8]:
#memory usage
for i,e in enumerate(vstup.state):
    try:
        vystup['memory_usage'][i] = vstup.state[i]['memory']['usage']
    except:
        pass
    

In [9]:
#created_at
for i, e in enumerate(vstup.created_at):
    vystup.created_at[i] = pd.to_datetime(vstup.created_at[i]).tz_convert('UTC')

In [10]:
#status
vystup.status = vstup.status

In [11]:
for i,e in enumerate(vstup.state):
    try:
        network = pd.DataFrame.from_dict(vstup.state[i]['network'])
    except:
        pass
    ips = []
    for j in network.loc['addresses']:
        for k in j:
            ips.append(k['address'])
    vystup.IP[i] = ips

In [12]:
vystup

,name,cpu_usage,memory_usage,created_at,status,IP
0,Mobile-applications-UI-test,0,52985856,2020-05-19 14:23:07+00:00,Running,"[172.18.0.1, fe80::42:a3ff:fe71:46d3, 10.17.17..."
1,docker-at,NaN,NaN,2020-03-01 00:03:08+00:00,Stopped,"[172.18.0.1, fe80::42:a3ff:fe71:46d3, 10.17.17..."
2,docker-logman-prod-env,0,763113472,2020-06-12 14:12:45+00:00,Running,"[127.0.0.1, ::1, 172.17.0.1, fe80::40e0:cff:fe..."
3,ecr-messenger-test,0,2099625984,2020-03-13 12:17:46+00:00,Running,"[fe80::6037:f4ff:fe7b:68c3, fe80::e4a6:61ff:fe..."
4,fastkafka-at-2006,0,76591104,2020-06-06 17:58:39+00:00,Running,"[10.17.164.107, fd87:1cf1:9a65:0:216:3eff:fe7d..."
5,ftp-test,0,57446400,2019-10-10 10:42:51+00:00,Running,"[10.17.170.231, fd87:1cf1:9a65:0:216:3eff:fe4d..."
6,genie-docker-test,0,632188928,2020-05-14 12:50:00+00:00,Running,"[10.17.161.11, fd87:1cf1:9a65:0:216:3eff:fec8:..."
7,gitlab-ci-runner-test,0,86732800,2020-06-08 10:40:16+00:00,Running,"[127.0.0.1, ::1, 172.17.0.1, fe80::42:daff:fe2..."
8,kafka-ui,0,190758912,2020-06-24 15:03:25+00:00,Running,"[fe80::6c10:4fff:fe0f:cbca, 172.17.0.1, fe80::..."
9,lmio-deploy-single-test,0,55820288,2020-05-25 10:45:43+00:00,Running,"[10.17.166.22, fd87:1cf1:9a65:0:216:3eff:fe9e:..."
